In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle

from pprint import pprint

from hab.airtable.airtable import AirtableHAB, airtable_oauth
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

import os
from nltk.corpus import stopwords

In [2]:
df = pd.concat([pd.read_pickle(file) for file in [x for x in os.listdir() if x.endswith(".pkl")]], ignore_index = True)

In [3]:
df.drop_duplicates("job_id", inplace = True)
df.reset_index(drop = True, inplace = True)

In [4]:
df

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,...,schedule_type,airtable_id,airtable_createdTime,contract_type,date_posted,source,country,remote_work,tech_skills,salary
0,Desarrollador de software de tiempo real,INTEGRASYS Group,España,a través de LinkedIn,"¿Te apasionan el Software, la Tecnología y el ...","[{'items': ['¿Te apasionan el Software, la Tec...","[{'link': 'http://www.integrasys-space.com/', ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 5 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Desarrollador/A Software,Buscojobs ES Premium,Zaragoza,a través de Trabajo.org,En qué consiste el trabajo:Desde Importante em...,[{'items': ['En qué consiste el trabajo:Desde ...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yL0EgU29mdH...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Desarrollador de Software para sistemas embebi...,IKERLAN,Mondragón,a través de Talent Clue,¡¡Estamos buscando a la próxima persona que pr...,[{'items': ['¡¡Estamos buscando a la próxima p...,"[{'link': 'http://www.ikerlan.es/', 'text': 'i...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[A tiempo completo],eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIFNvZn...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Desarrollador de software C/C++,atSistemas,España,a través de LinkedIn,"En la recta final del primer semestre de 2023,...",[{'items': ['En la recta final del primer seme...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 4 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Desarrollador Software,UPTECH SENSING,San Sebastián,a través de Indeed,UPTECH SENSING busca personal para cubrir la p...,[{'items': ['UPTECH SENSING busca personal par...,"[{'link': 'http://www.uptech-sensing.com/', 't...",NaN,[A tiempo completo],eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIFNvZnR3YX...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21304,Regulatory Data Analyst,BME | Bolsas y Mercados Españoles,Madrid,a través de LinkedIn,Job description:\n\nBME is the infrastructure ...,[{'items': ['Job description: BME is the infr...,"[{'link': 'http://www.bolsasymercados.es/', 't...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 20 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJSZWd1bGF0b3J5IERhdGEgQW5hbH...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21305,Corporate Data Analyst,Leroy Merlin - España,Alcobendas,a través de LinkedIn,Centro de Servicios a Tienda y Desarrollo de N...,[{'items': ['Centro de Servicios a Tienda y De...,"[{'link': 'http://www.leroymerlin.es/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[A tiempo completo],eyJqb2JfdGl0bGUiOiJDb3Jwb3JhdGUgRGF0YSBBbmFseX...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21306,Master Data Analyst,SGS,Madrid,a través de LinkedIn,Company Description\n\nWe are SGS – the world’...,[{'items': ['Company Description We are SGS –...,"[{'link': 'http://www.sgs.com/', 'text': 'sgs....",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 15 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJNYXN0ZXIgRGF0YSBBbmFseXN0Ii...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21307,Prácticas Data Analyst,Fintonic,Madrid,a través de LinkedIn,Fintonic es una start-up en el sector fintech ...,[{'items': ['Fintonic es una start-up en el se...,"[{'link': 'http://www.fintonic.com/', 'text': ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 11 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQcsOhY3RpY2FzIERhdGEgQW5hbH...,...,A tiempo completo

In [5]:
df_new = df.copy()

# Clean "via" columns
df_new["via"] = df_new["via"].apply(lambda x : clean.clean_source(x))
df_new["via"] = df_new["via"].apply(lambda x : clean.clean_regular_brackets(x) if type(x) == str else x)

# Clean "location" column
df_new["location"] = df_new["location"].apply(lambda x : clean.clean_regular_brackets(x) if type(x) == str else x)

In [6]:
df_new["location"].value_counts()

Madrid         8519
España         3037
Barcelona      1752
Valencia       1446
Zaragoza        878
               ... 
Abadiano          1
Jauja             1
Llodio            1
Veriña            1
Spanyol (+5       1
Name: location, Length: 447, dtype: int64

In [7]:
df_new

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,...,schedule_type,airtable_id,airtable_createdTime,contract_type,date_posted,source,country,remote_work,tech_skills,salary
0,Desarrollador de software de tiempo real,INTEGRASYS Group,España,LinkedIn,"¿Te apasionan el Software, la Tecnología y el ...","[{'items': ['¿Te apasionan el Software, la Tec...","[{'link': 'http://www.integrasys-space.com/', ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 5 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Desarrollador/A Software,Buscojobs ES Premium,Zaragoza,Trabajo.org,En qué consiste el trabajo:Desde Importante em...,[{'items': ['En qué consiste el trabajo:Desde ...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yL0EgU29mdH...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Desarrollador de Software para sistemas embebi...,IKERLAN,Mondragón,Talent Clue,¡¡Estamos buscando a la próxima persona que pr...,[{'items': ['¡¡Estamos buscando a la próxima p...,"[{'link': 'http://www.ikerlan.es/', 'text': 'i...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[A tiempo completo],eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIFNvZn...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Desarrollador de software C/C++,atSistemas,España,LinkedIn,"En la recta final del primer semestre de 2023,...",[{'items': ['En la recta final del primer seme...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 4 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Desarrollador Software,UPTECH SENSING,San Sebastián,Indeed,UPTECH SENSING busca personal para cubrir la p...,[{'items': ['UPTECH SENSING busca personal par...,"[{'link': 'http://www.uptech-sensing.com/', 't...",NaN,[A tiempo completo],eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIFNvZnR3YX...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21304,Regulatory Data Analyst,BME | Bolsas y Mercados Españoles,Madrid,LinkedIn,Job description:\n\nBME is the infrastructure ...,[{'items': ['Job description: BME is the infr...,"[{'link': 'http://www.bolsasymercados.es/', 't...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 20 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJSZWd1bGF0b3J5IERhdGEgQW5hbH...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21305,Corporate Data Analyst,Leroy Merlin - España,Alcobendas,LinkedIn,Centro de Servicios a Tienda y Desarrollo de N...,[{'items': ['Centro de Servicios a Tienda y De...,"[{'link': 'http://www.leroymerlin.es/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[A tiempo completo],eyJqb2JfdGl0bGUiOiJDb3Jwb3JhdGUgRGF0YSBBbmFseX...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21306,Master Data Analyst,SGS,Madrid,LinkedIn,Company Description\n\nWe are SGS – the world’...,[{'items': ['Company Description We are SGS –...,"[{'link': 'http://www.sgs.com/', 'text': 'sgs....",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 15 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJNYXN0ZXIgRGF0YSBBbmFseXN0Ii...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21307,Prácticas Data Analyst,Fintonic,Madrid,LinkedIn,Fintonic es una start-up en el sector fintech ...,[{'items': ['Fintonic es una start-up en el se...,"[{'link': 'http://www.fintonic.com/', 'text': ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 11 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQcsOhY3RpY2FzIERhdGEgQW5hbH...,...,A tiempo completo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
STOPWORDS = set(stopwords.words("spanish") + stopwords.words("english")) + ["•", "-", "/", "¿", "?", "!", "¡", ":", "·", "–", "&",
                                                                            ".", ",", "●", "•..."]

TypeError: unsupported operand type(s) for +: 'set' and 'list'

In [ ]:
def clean_stopwords(string):

    return " ".join([x for x in string.split() if x not in STOPWORDS])

In [ ]:
df["description2"] = df["description"].apply(lambda x : clean_stopwords(x))

In [ ]:
textos = " ".join(df["description2"]).lower()

from collections import Counter

counter = Counter(textos.split())

In [ ]:
counter.most_common(1000)

In [ ]:
from textblob import TextBlob

blob = TextBlob(textos)

grams = blob.ngrams(2)

In [ ]:
lista_grams = [list(x) for x in grams]

In [ ]:
counter_prueba = Counter([" ".join(x) for x in lista_grams])

counter_prueba.most_common(1000)

In [2]:
import pandas as pd
df1 = pd.read_csv("C:/Users/danie/OneDrive/Desktop/Spain-All data.csv")

In [5]:
df1.drop(df1.index).to_csv("columnas.csv", index = False)

In [8]:
import seaborn as sns

sns.load_dataset.iris_load()

AttributeError: 'function' object has no attribute 'iris_load'